In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
from tqdm import tqdm
import os
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from torchvision.models.inception import InceptionOutputs
import time

USE_GPU = True
dtype = torch.float32 # We will be using float

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = -1
print('using device:', device)

dropout_flag = True
dropout_prob = 0.5
if dropout_flag:
    path_to_store_the_plots = 'plots/conv2d_with_dropout/'
    
else:
    path_to_store_the_plots = 'plots/conv2d/'

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch

from sklearn.preprocessing import LabelEncoder

# Define a transform without normalization
transform = transforms.Compose([
    transforms.ToTensor()  # Convert NumPy array to PyTorch tensor
])

class NumpyDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X  # Keep as NumPy array
        self.transform = transform  # Store the transform
        
        # Convert string labels into numeric using LabelEncoder
        self.label_encoder = LabelEncoder()
        self.Y = [str(y[0]) for y in Y]  # Ensure Y is a list of strings
        self.classes = list(sorted(set(self.Y)))  # Unique class names
        self.Y = self.label_encoder.fit_transform(self.Y)  # Convert to numeric labels

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]  # NumPy array
        y = self.Y[idx]  # Numeric label (integer)

        # Apply the transform to the image if provided
        if self.transform:
            x = self.transform(x)

        return x, y

# Define the batch size
batch_size = 64

dataset_path = "../../../sign_datasets/27-class-sign-language-dataset/"
path_to_store_the_plots = path_to_store_the_plots + dataset_path.split('/')[-1]

# Check if the directory exists, and create it if it doesn't
os.makedirs(path_to_store_the_plots, exist_ok=True)

print(f"Plots will be stored in: {path_to_store_the_plots}")

# Step 1: Load the .npy files
X = np.load(dataset_path + "X.npy")  # Shape: (Samples, H, W, C), dtype: float
Y = np.load(dataset_path + "Y.npy")  # Shape: (Samples, 1), dtype: str

# Step 3: Instantiate the dataset
# Instantiate the dataset with the transform
dataset = NumpyDataset(X, Y, transform=transform)

# Step 4: Split the dataset into train, validation, and test sets
total_samples = len(dataset)
train_size = int(0.7 * total_samples)  # 70% for training
val_size = int(0.2 * total_samples)    # 20% for validation
test_size = total_samples - train_size - val_size  # Remaining for testing

# Perform the split
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Step 5: Print the sizes of each split
print(f"Total samples: {total_samples}")
print(f"Training set size: {train_size}")
print(f"Validation set size: {val_size}")
print(f"Test set size: {test_size}")

In [ ]:
# Already computed mean and std
computed_mean = [0.5108343362808228, 0.47160401940345764, 0.43902790546417236]
computed_std = [0.5867133736610413, 0.5466195344924927, 0.5143911242485046]

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),       # Convert images to PyTorch tensors
    transforms.Normalize(mean=computed_mean, std=computed_std)  # Normalize
])

# Create the dataset
dataset = NumpyDataset(X, Y, transform=transform)

# Define class names based on unique labels in Y
class_names = list(set([str(y[0]) for y in Y]))
print("Class names:", class_names)

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each split
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes for verification
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

len_train_loader = len(train_loader)
len_val_loader = len(val_loader)
len_test_loader = len(test_loader)

# Print DataLoader sizes for verification
print(f"Train DataLoader size: {len(train_loader)}")
print(f"Validation DataLoader size: {len(val_loader)}")
print(f"Test DataLoader size: {len(test_loader)}")

In [ ]:
# Access the first image in the dataset
first_image, _ = train_loader.dataset[0]

# Extract dimensions
width = first_image.shape[1]
height = first_image.shape[2]

print(f"Width: {width}, Height: {height}")

In [5]:
def check_accuracy_part(loader, model, return_acc=False):

    # Determine the split type
    if len(loader) == len_train_loader:
        split = 'train'
    elif len(loader) == len_val_loader:
        split = 'val'
    elif len(loader) == len_test_loader:
        split = 'test'

    print(f'Checking accuracy on the {split} set')

    # Init counters and placeholders
    num_correct = 0
    num_samples = 0
    all_preds = []
    all_labels = []

    # Set model to evaluation mode
    model.eval()

    # Disable gradient computation as we don't train
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # Move to device, e.g., GPU
            y = y.to(device=device, dtype=torch.long)  # Use torch.long for labels

            # Forward pass
            scores = model(x)
            _, preds = scores.max(1)  # Get predicted class -> highest score

            # Collect all predictions and labels for confusion matrix
            all_preds.append(preds.cpu())  # Append to list, move to CPU for numpy
            all_labels.append(y.cpu())    # Append to list, move to CPU for numpy

            # Count correct predictions and total samples
            correct_predictions = preds == y  # True if correct - False otherwise
            num_correct += correct_predictions.sum().item()
            num_samples += preds.size(0)

    # Compute accuracy
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

    # For test set: plot confusion matrix
    if split == 'test':
        all_preds = torch.cat(all_preds).numpy()  # Flatten and convert to numpy
        all_labels = torch.cat(all_labels).numpy()

        # Compute confusion matrix
        conf_matrix = confusion_matrix(all_labels, all_preds)
        np.savetxt(path_to_store_the_plots + '/confusion_matrix.txt', conf_matrix, fmt='%d')

        # Plot confusion matrix
        plt.figure(figsize=(20, 20))
        sns.heatmap(
            conf_matrix,
            annot=True,
            fmt='d',
            cmap='Greens',
            linewidths=0.5,
            xticklabels=class_names,
            yticklabels=class_names
        )
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix with Actual Class Labels')
        plt.savefig(path_to_store_the_plots + '/confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.show()

    if return_acc:
        return 100 * acc

In [6]:
def train_part(model, optimizer, epochs=1, scheduler = None, return_acc = False, check_train_set = False, check_test_set = False, early_stopping = False, patience = 5):
    
    # List to store validation and train accuracies at each epoch where needed
    val_accs = []
    train_accs = []
    losses = []
    val_losses = [] 
    best_val_loss = float('inf')  # Initialize with a high value
    
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t+1 % print_every == 0 and print_every!=-1 :
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part(val_loader, model)
                print()
        
        losses.append(loss.item())
        print(f"Training Loss after epoch {e+1}: {loss.item()}")

        # Step the scheduler after each epoch to adjust the learning rate if needed
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                # If the scheduler is of type ReduceLROnPlateau, use validation loss for adjustment
                model.eval()  # Set model to evaluation mode
                val_loss = 0.0  # Initialize validation loss accumulator
                with torch.no_grad():  # No need to compute gradients during validation
                    for data, target in val_loader:
                        data, target = data.to(device=device, dtype=dtype), target.to(device=device, dtype=torch.long)
                        output = model(data)
                        loss = F.cross_entropy(output, target)
                        val_loss += loss.item()  # Accumulate the validation loss

                val_loss /= len(val_loader)  # Average the validation loss
                scheduler.step(val_loss)  # Step the scheduler based on the validation loss
            else:
                scheduler.step()  # Step the scheduler for other types of schedulers

        # Calculate validation loss for the epoch
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device=device, dtype=dtype), target.to(device=device, dtype=torch.long)
                output = model(data)
                loss = F.cross_entropy(output, target)
                val_loss += loss.item()
        val_loss /= len(val_loader)  # Average validation loss
        val_losses.append(val_loss)  # Append validation loss to list

        # After each epoch, check accuracies
        val_acc = check_accuracy_part(val_loader, model, return_acc=True)
        val_accs.append(val_acc)
        print(f"Validation Accuracy after epoch {e+1}: {val_acc:.2f}%")
        print(f"Validation Loss after epoch {e+1}: {val_loss:.2f}")
        
        # Return accuracies if requested
        if check_train_set:
            train_acc = check_accuracy_part(train_loader, model, return_acc=True)
            train_accs.append(train_acc)
            print(f"Training Accuracy after epoch {e+1}: {train_acc:.2f}%")

        if early_stopping:
            # Early stopping logic
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f"Early stopping at epoch {e+1}!")
                    break

    # Return accuracies if requested
    if return_acc:
        if check_train_set and check_test_set:
            test_acc = check_accuracy_part(test_loader, model, return_acc=True)  # Check test set accuracy
            return val_accs, train_accs, test_acc, losses, val_losses  # Return the most recent validation accuracy and the test accuracy
        if check_train_set:  # If requested, return both validation and training accuracies
            return val_accs, train_accs, losses, val_losses
        if check_test_set:  # If requested, return the validation accuracy and test accuracy
            test_acc = check_accuracy_part(test_loader, model, return_acc=True)  # Check test set accuracy
            return val_accs, test_acc, losses, val_losses  # Return the most recent validation accuracy and the test accuracy
        return val_accs, losses, val_losses  # Return only validation accuracies if no other set (train or test) is specified

In [ ]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-3

in_channel = 3
num_classes = len(class_names)

if dropout_flag:
    print('Train With Dropout.')

    # Define the model using nn.Sequential
    model = nn.Sequential(
        nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2),
        nn.BatchNorm2d(channel_1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Add pooling to reduce spatial dimensions
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
        nn.BatchNorm2d(channel_2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Add pooling to reduce spatial dimensions
        nn.Flatten(),
        nn.Dropout(dropout_prob),
        nn.Linear(channel_2 * (width // 4) * (height // 4), num_classes)  # Adjust input size due to pooling
    )
    early_stopping = True

else:
    print('Train Without Dropout.')
    # Define the model using nn.Sequential
    model = nn.Sequential(
        nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2), # First conv layer with Relu
        nn.ReLU(),
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1), # Second conv layer with Relu
        nn.ReLU(),                                             
        nn.Flatten(), # Flatten the output
        nn.Linear(channel_2 * width * height, num_classes) # Fully-connected layer, (channel_2 * 50 * 50) -> the number of features coming into the fully connected layer.
    )
    early_stopping = False
    
# You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

# Start the timer
start_time = time.time()
val_accs, train_accs, test_acc, losses, val_losses = train_part(model, optimizer,epochs=20, return_acc=True, check_train_set = True, check_test_set = True, early_stopping=early_stopping)
# Stop the timer
elapsed_time = time.time() - start_time

# Print the validation and test accuracies
print()
print('########### Final Results ###########')
print("Elapsed Time: {:.2f} seconds".format(elapsed_time))
print("Validation Accuracy:", val_accs[-1])
print("Test Accuracy:", test_acc)

In [ ]:
# Plot accuracy
epochs = range(len(val_accs))
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_accs, 'r', label='Training Accuracy')
plt.plot(epochs, val_accs, 'b', label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig(path_to_store_the_plots + '/Accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot loss
plt.figure(figsize=(12, 6))
plt.plot(epochs, losses, 'r', label='Training Loss')
plt.plot(epochs, val_losses, 'b', label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.savefig(path_to_store_the_plots + '/Loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Ensure model is in evaluation mode
model.eval()

# Initialize placeholders for features and labels
features, labels_list = [], []

# Disable gradient computation
with torch.no_grad():
    for inputs, labels in val_loader:
        # Move data to the appropriate device
        inputs = inputs.to(device)
        
        # Forward pass to extract features
        outputs = model(inputs)
        
        # Handle Inception model outputs
        if isinstance(outputs, InceptionOutputs):
            outputs = outputs.logits
        
        # Collect features and labels
        features.append(outputs.cpu().numpy())  # Move to CPU and convert to numpy
        labels_list.extend(labels.numpy())  # Add labels to list

# Concatenate all features
features = np.concatenate(features, axis=0)

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)
features_2d = tsne.fit_transform(features)

# Visualize with a scatter plot
plt.figure(figsize=(16, 16))
scatter = sns.scatterplot(
    x=features_2d[:, 0],
    y=features_2d[:, 1],
    hue=labels_list,
    palette=sns.color_palette("hsv", len(dataset.classes)),
    legend='full',
    alpha=0.7
)

# Add labels and title
plt.xlabel('t-SNE Feature 1')
plt.ylabel('t-SNE Feature 2')
plt.title('t-SNE Visualization of Feature Representations')

# Add a color bar for better visualization
scatter.figure.colorbar(scatter.collections[0], label='Class Labels')
plt.savefig(path_to_store_the_plots + '/t-SNE.png', dpi=300, bbox_inches='tight')
plt.show()